# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = os.path.join('..', 'output_data', 'cities.csv')

city_df = pd.read_csv(csv_file)
city_df

,City ID,City,Country,Lat,Lng,Temp,Humidity,Clouds,Wind Speed
0,2206939,Bluff,NZ,-46.60,168.33,57.65,60,35,25.19
1,5122534,Jamestown,US,42.10,-79.24,65.52,63,1,4.70
2,5364271,Laguna,US,38.42,-121.42,93.24,27,1,11.63
3,5983720,Iqaluit,CA,63.75,-68.51,44.29,81,20,5.82
4,2629833,Husavik,IS,66.04,-17.34,53.60,76,90,9.17
...,...,...,...,...,...,...,...,...,...
556,2126785,Belaya Gora,RU,68.53,146.42,51.26,54,35,12.28
557,727987,Preslav,BG,43.17,26.82,64.00,90,0,1.79
558,3702577,Portobelo,PA,9.55,-79.65,81.95,78,98,7.00
559,4176380,Venice,US,27.10,-82.45,77.00,94,40,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_df = city_df[(city_df['Temp'] >= 75) & (city_df['Temp'] <= 90)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Clouds'] <= 10]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
hotel_df = ideal_df
hotel_df

,City ID,City,Country,Lat,Lng,Temp,Humidity,Clouds,Wind Speed
37,3516522,Soto la Marina,MX,23.77,-98.22,89.01,68,6,10.00
60,2219235,Ubari,LY,26.59,12.78,85.87,19,0,8.12
63,88533,Awjilah,LY,29.11,21.29,80.64,33,0,7.54
88,164947,Salamiyah,SY,35.01,37.05,77.40,27,0,5.66
175,6949678,Taitung City,TW,22.76,121.14,84.20,70,7,3.36
187,2063039,Port Keats,AU,-14.25,129.55,81.90,51,0,9.57
204,108648,Qurayyat,SA,31.33,37.34,86.00,19,0,4.70
236,3456248,Nortelândia,BR,-14.45,-56.80,81.99,21,0,3.02
340,298033,Viranşehir,TR,37.24,39.76,87.71,12,0,9.82
349,6620339,Karratha,AU,-20.74,116.85,80.33,28,3,6.89


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name'] = ""
hotel_df

,City ID,City,Country,Lat,Lng,Temp,Humidity,Clouds,Wind Speed,Hotel Name
37,3516522,Soto la Marina,MX,23.77,-98.22,89.01,68,6,10.00,
60,2219235,Ubari,LY,26.59,12.78,85.87,19,0,8.12,
63,88533,Awjilah,LY,29.11,21.29,80.64,33,0,7.54,
88,164947,Salamiyah,SY,35.01,37.05,77.40,27,0,5.66,
175,6949678,Taitung City,TW,22.76,121.14,84.20,70,7,3.36,
187,2063039,Port Keats,AU,-14.25,129.55,81.90,51,0,9.57,
204,108648,Qurayyat,SA,31.33,37.34,86.00,19,0,4.70,
236,3456248,Nortelândia,BR,-14.45,-56.80,81.99,21,0,3.02,
340,298033,Viranşehir,TR,37.24,39.76,87.71,12,0,9.82,
349,6620339,Karratha,AU,-20.74,116.85,80.33,28,3,6.89,


In [7]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City ID,City,Country,Lat,Lng,Temp,Humidity,Clouds,Wind Speed,Hotel Name
37,3516522,Soto la Marina,MX,23.77,-98.22,89.01,68,6,10.00,Hotel Los Generales INN
60,2219235,Ubari,LY,26.59,12.78,85.87,19,0,8.12,فندق أوباري
63,88533,Awjilah,LY,29.11,21.29,80.64,33,0,7.54,NaN
88,164947,Salamiyah,SY,35.01,37.05,77.40,27,0,5.66,Resort Safari Souria منتجع سفاري سورية
175,6949678,Taitung City,TW,22.76,121.14,84.20,70,7,3.36,Sheraton Taitung Hotel
187,2063039,Port Keats,AU,-14.25,129.55,81.90,51,0,9.57,TDC Village (Hundred Man Camp)
204,108648,Qurayyat,SA,31.33,37.34,86.00,19,0,4.70,The life of the Knights Hotel Suites
236,3456248,Nortelândia,BR,-14.45,-56.80,81.99,21,0,3.02,Hotel Karysmã
340,298033,Viranşehir,TR,37.24,39.76,87.71,12,0,9.82,Tella Hotel
349,6620339,Karratha,AU,-20.74,116.85,80.33,28,3,6.89,Best Western Plus The Ranges Karratha


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))